In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [3]:
from tqdm.auto import tqdm
import numpy as np
import jax.numpy as jnp

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

from utils_HD import MatConvert, MMDu, TST_MMD_u

In [4]:
np.random.seed(819)
torch.manual_seed(819)
torch.cuda.manual_seed(819)
torch.backends.cudnn.deterministic = True
is_cuda = True

In [5]:
# Parameter Settings
n_epochs = 1000
batch_size = 100
lr = 0.0002
img_size = 64
channels = 3
n = 1000

In [6]:
dtype = torch.float
device = torch.device("cuda:0")
cuda = True if torch.cuda.is_available() else False
N_per = 100 # permutation times
alpha = 0.05 # test threshold
N1 = n # number of samples in one set
K = 10 # number of trails
J = 1 # number of test locations
N = 100 # number of test sets
N_f = 100.0 # number of test sets (float)

In [7]:
# Naming variables
ep_OPT = np.zeros([K])
s_OPT = np.zeros([K])
s0_OPT = np.zeros([K])
T_org_OPT = torch.zeros([K,J,3,64,64]) # Record test locations obtained by MMD-D
DK_Results = np.zeros([1,K])

In [8]:
# Define the deep network for MMD-D
class Featurizer(nn.Module):
    def __init__(self):
        super(Featurizer, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0)] #0.25
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
            # *discriminator_block(128, 256), 
            
        )

        # The height and width of downsampled image
        ds_size = img_size // 2 ** 4
        self.adv_layer = nn.Sequential(
            nn.Linear(128 * ds_size ** 2, 300))

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        feature = self.adv_layer(out)

        return feature

In [9]:
# Configure data loader
dataset_test = datasets.CIFAR10(root='/data4/oldrain123/C2ST/data/cifar_data/cifar10', download=True,train=False,
                           transform=transforms.Compose([
                               transforms.Resize(img_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=10000,
                                             shuffle=True, num_workers=1)
# Obtain CIFAR10 images
for i, (imgs, Labels) in enumerate(dataloader_test):
    data_all = imgs
    label_all = Labels
Ind_all = np.arange(len(data_all))

# Obtain CIFAR10.1 images
data_new = np.load('/data4/oldrain123/C2ST/data/cifar10_1/cifar10.1_v4_data.npy')
data_T = np.transpose(data_new, [0,3,1,2])
ind_M = np.random.choice(len(data_T), len(data_T), replace=False)
data_T = data_T[ind_M]
TT = transforms.Compose([transforms.Resize(img_size),transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trans = transforms.ToPILImage()
data_trans = torch.zeros([len(data_T),3,img_size,img_size])
data_T_tensor = torch.from_numpy(data_T)
for i in range(len(data_T)):
    d0 = trans(data_T_tensor[i])
    data_trans[i] = TT(d0)
Ind_v4_all = np.arange(len(data_T))

Files already downloaded and verified


In [10]:
# Repeat experiments K times (K = 10) and report average test power (rejection rate)
for kk in tqdm(range(K)):
    torch.manual_seed(kk * 19 + N1)
    torch.cuda.manual_seed(kk * 19 + N1)
    np.random.seed(seed=1102 * (kk + 10) + N1)
    
    # Initialize deep networks for MMD-D (called featurizer)
    featurizer = Featurizer()
    
    # Initialize parameters
    epsilonOPT = torch.log(MatConvert(np.random.rand(1) * 10 ** (-10), device, dtype))
    epsilonOPT.requires_grad = True
    sigmaOPT = MatConvert(np.ones(1) * np.sqrt(2 * 32 * 32), device, dtype)
    sigmaOPT.requires_grad = True
    sigma0OPT = MatConvert(np.ones(1) * np.sqrt(0.005), device, dtype)
    sigma0OPT.requires_grad = True
    if cuda:
        featurizer.cuda()

    # Collect CIFAR10 images
    Ind_tr = np.random.choice(len(data_all), N1, replace=False)
    Ind_te = np.delete(Ind_all, Ind_tr)
    train_data = []
    for i in Ind_tr:
       train_data.append([data_all[i], label_all[i]])

    dataloader = torch.utils.data.DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
    )

    # Collect CIFAR10.1 images
    np.random.seed(seed=819 * (kk + 9) + N1)
    Ind_tr_v4 = np.random.choice(len(data_T), N1, replace=False)
    Ind_te_v4 = np.delete(Ind_v4_all, Ind_tr_v4)
    New_CIFAR_tr = data_trans[Ind_tr_v4]
    New_CIFAR_te = data_trans[Ind_te_v4]

    # Initialize optimizers
    optimizer_F = torch.optim.Adam(list(featurizer.parameters()) + [epsilonOPT] + [sigmaOPT] + [sigma0OPT], lr=lr)

    Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

    # ----------------------------------------------------------------------------------------------------
    #  Training deep networks for MMD-D (called featurizer)
    # ----------------------------------------------------------------------------------------------------
    np.random.seed(seed=1102)
    torch.manual_seed(1102)
    torch.cuda.manual_seed(1102)
    for epoch in tqdm(range(n_epochs)):
        for i, (imgs, _) in enumerate(dataloader):
            if True:
                ind = np.random.choice(N1, imgs.shape[0], replace=False)
                Fake_imgs = New_CIFAR_tr[ind]
                # Adversarial ground truths
                valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
                fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

                # Configure input
                real_imgs = Variable(imgs.type(Tensor))
                Fake_imgs = Variable(Fake_imgs.type(Tensor))
                X = torch.cat([real_imgs, Fake_imgs], 0)
                Y = torch.cat([valid, fake], 0).squeeze().long()

                # ------------------------------
                #  Train deep network for MMD-D
                # ------------------------------
                # Initialize optimizer
                optimizer_F.zero_grad()
                # Compute output of deep network
                modelu_output = featurizer(X)
                # Compute epsilon, sigma and sigma_0
                ep = torch.exp(epsilonOPT) / (1 + torch.exp(epsilonOPT))
                sigma = sigmaOPT ** 2
                sigma0_u = sigma0OPT ** 2
                # Compute Compute J (STAT_u)
                DK_TEMP = MMDu(modelu_output, imgs.shape[0], X.view(X.shape[0],-1), sigma, sigma0_u, ep, complete=False)
                dk_hsic_xx = torch.from_numpy(np.array(DK_TEMP[3])).to(torch.float32)
                dk_hsic_yy = torch.from_numpy(np.array(DK_TEMP[4])).to(torch.float32)
                dk_hsic_xy = torch.from_numpy(np.array(DK_TEMP[5])).to(torch.float32)
                
                dk_mmd = -1 * (DK_TEMP[0])
                dk_mmd_std = torch.sqrt(DK_TEMP[1] + 10 ** (-8))
                
                DK_STAT_u = torch.div(dk_mmd, dk_mmd_std) + (dk_hsic_xx + dk_hsic_yy - dk_hsic_xy)
                if (epoch+1) % 100 == 0:
                    print("-" * 50)
                    print("epoch : ",epoch+1)
                    print("dk mmd: ", -1 * dk_mmd.item(), "dk mmd_std: ", dk_mmd_std.item(), "dk statistic: ",
                    -1 * DK_STAT_u.item())
                    
                # Compute gradient
                DK_STAT_u.backward()                
                # Update weights using gradient descent
                optimizer_F.step()
            else:
                break

    # Run two-sample test on the training set
    # Fetch training data
    s1 = data_all[Ind_tr]
    s2 = data_trans[Ind_tr_v4]
    S = torch.cat([s1.cpu(), s2.cpu()], 0).cuda()
    Sv = S.view(2 * N1, -1)
    # Run two-sample test (MMD-D) on the training set
    # dk_h_u, dk_threshold_u, dk_mmd_value_u = TST_MMD_u(featurizer(S), N_per, N1, Sv, sigma, sigma0_u, ep, alpha, device, dtype, complete=False)
    # com_h_u, com_threshold_u, com_mmd_value_u = TST_MMD_u(featurizer(S), N_per, N1, Sv, sigma, sigma0_u, ep, alpha, device, dtype, complete=True)

    # Record best epsilon, sigma and sigma_0
    ep_OPT[kk] = ep.item()
    s_OPT[kk] = sigma.item()
    s0_OPT[kk] = sigma0_u.item()

    # Compute test power of MMD-D and baselines
    DK_H_u = np.zeros(N)

    np.random.seed(1102)
    dk_count_u = 0

    for k in tqdm(range(N)):
        # Fetch test data
        np.random.seed(seed=1102 * (k + 1) + N1)
        data_all_te = data_all[Ind_te]
        N_te = len(data_trans)-N1
        Ind_N_te = np.random.choice(len(Ind_te), N_te, replace=False)
        s1 = data_all_te[Ind_N_te]
        s2 = data_trans[Ind_te_v4]
        S = torch.cat([s1.cpu(), s2.cpu()], 0).cuda()
        Sv = S.view(2 * N_te, -1)
        # MMD-D
        dk_h_u, dk_threshold_u, dk_mmd_value_u = TST_MMD_u(featurizer(S), N_per, N_te, Sv, sigma, sigma0_u, ep, alpha, device, dtype, complete=False)

        # Gather results
        dk_count_u = dk_count_u + dk_h_u
        print("\r","MMD-DK:", dk_count_u, "MMD: ", dk_mmd_value_u, end="")
        DK_H_u[k] = dk_h_u

    # Print test power of MMD-D and baselines
    print("DK Reject rate_u: ", DK_H_u.sum() / N_f)
    DK_Results[0, kk] = DK_H_u.sum() / N_f
    print(f"Test Power of DK ({K} times): ")
    print(f"{DK_Results}")
    print(f"Average Test Power of DK ({K} times): ")
    print("MMD-D: ", (DK_Results.sum(1) / (kk + 1))[0])

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

/tmp/ipykernel_1126663/2714037299.py:57: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392035629/work/torch/csrc/tensor/python_tensor.cpp:83.)
  valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
CUDA backend failed to initialize: Found cuPTI version 18, but JAX was built against version 20, which is newer. The copy of cuPTI that is installed must be at least as new as the version against which JAX was built. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


--------------------------------------------------
epoch :  100
dk mmd:  0.009275173768401146 dk mmd_std:  0.004235982429236174 dk statistic:  2.179488182067871
--------------------------------------------------
epoch :  100
dk mmd:  0.009926260448992252 dk mmd_std:  0.0036966900806874037 dk statistic:  2.6750881671905518
--------------------------------------------------
epoch :  100
dk mmd:  0.010025662370026112 dk mmd_std:  0.004053354728966951 dk statistic:  2.4632937908172607
--------------------------------------------------
epoch :  100
dk mmd:  0.010198758915066719 dk mmd_std:  0.0036343138199299574 dk statistic:  2.7961575984954834
--------------------------------------------------
epoch :  100
dk mmd:  0.009643965400755405 dk mmd_std:  0.0030855198856443167 dk statistic:  3.115452527999878
--------------------------------------------------
epoch :  100
dk mmd:  0.009220937266945839 dk mmd_std:  0.002899167826399207 dk statistic:  3.170462131500244
----------------------------

  0%|          | 0/100 [00:00<?, ?it/s]

...

MMDs:  [-1.25577208e-06 -2.57929787e-06 -5.02052717e-06 -8.49657226e-06
  3.63402069e-06  7.93195795e-06 -5.31703699e-06  1.51604181e-05
  7.05569983e-06  4.13111411e-06 -4.35160473e-07  3.13948840e-06
 -6.60971273e-06  5.05161006e-06 -3.87232285e-06 -1.43062789e-05
 -3.36056110e-06  8.41566361e-07  2.53140461e-05  3.77453398e-06
  6.06407411e-07 -5.22180926e-06 -2.55159102e-06 -1.22305937e-06
 -6.88142609e-06  1.97542831e-05 -3.51632480e-06  2.48289434e-05
 -1.36275776e-06  1.53778819e-05  5.31307887e-06 -2.73785554e-06
  9.91672277e-06  3.97115946e-06  6.43194653e-07  2.53308099e-06
  4.99596354e-06  3.23669519e-06  1.01525802e-06  2.20040092e-05
  2.74522463e-05 -2.06031837e-06  2.38453504e-06  8.63592140e-06
  1.32108107e-06 -7.82660209e-07  4.35102265e-06  1.80462375e-05
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  

  0%|          | 0/1000 [00:00<?, ?it/s]

--------------------------------------------------
epoch :  100
dk mmd:  0.013018261641263962 dk mmd_std:  0.004431330598890781 dk statistic:  2.9276676177978516
--------------------------------------------------
epoch :  100
dk mmd:  0.012294463813304901 dk mmd_std:  0.003352038562297821 dk statistic:  3.6576483249664307
--------------------------------------------------
epoch :  100
dk mmd:  0.011401613242924213 dk mmd_std:  0.003723236732184887 dk statistic:  3.0521576404571533
--------------------------------------------------
epoch :  100
dk mmd:  0.010555597953498363 dk mmd_std:  0.003261633450165391 dk statistic:  3.2261579036712646
--------------------------------------------------
epoch :  100
dk mmd:  0.01042219065129757 dk mmd_std:  0.00412119971588254 dk statistic:  2.5187885761260986
--------------------------------------------------
epoch :  100
dk mmd:  0.010001448914408684 dk mmd_std:  0.0039781262166798115 dk statistic:  2.5039687156677246
-----------------------------

  0%|          | 0/100 [00:00<?, ?it/s]

MMDs:  [-1.71570573e-05  2.94563361e-05 -1.97917689e-05 -6.75115734e-06
 -2.98209488e-06 -8.35210085e-06 -1.88434497e-05 -4.98769805e-06
 -1.71368010e-05  1.68115366e-05  3.62470746e-06  4.47407365e-06
  3.17906961e-05 -1.67035032e-05  1.84951350e-05 -3.38954851e-06
 -4.28175554e-06 -1.14252325e-05 -4.26545739e-06  2.24611722e-05
 -1.69118866e-05  1.42478384e-05  1.32694840e-05  5.79794869e-06
  4.70187515e-05 -4.55998816e-06  1.03423372e-06 -9.10903327e-06
 -1.23165082e-05  7.71437772e-06  6.52414747e-06 -2.78395601e-06
 -1.85449608e-05 -3.35765071e-06  1.54820736e-05 -7.57328235e-06
 -6.38980418e-06 -1.41579658e-05 -1.41165219e-06 -7.28294253e-06
  1.36904418e-07 -9.39820893e-06  9.19052400e-06 -1.69449486e-05
 -2.30551232e-05  1.51386485e-06  6.69690780e-06  1.96609180e-05
  3.51700000e-05  2.60719098e-05  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  

#### Epoch 20 / lr: 0.0005
[[0.23 0.94 0.25 0.37 0.6  0.1  0.36 0.31 0.23 0.4 ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]]
Average Test Power of Baselines (10 times): 
MMD-D:  0.379